In [1]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

from german_parser.util.const import CONSTS
from german_parser.util.dataloader import TigerDatasetGenerator
import dill as pickle

import multiprocess

Using cache found in /mnt/d/torch_data/hub/huggingface_pytorch-transformers_main


In [2]:
character_flag_generators = [
            lambda c: int(c.isupper()),
            lambda c: int(c.lower() in ["ä", "ö", "ü", "ß"]),
            lambda c: int(c.isdigit()),
            lambda c: int(c in r"""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~""")
        ]

In [3]:
iterations = set(range(30)) - {3,12}

In [4]:
def get_required_vars(character_flag_generators):
    dataset_generator = TigerDatasetGenerator(f"{CONSTS['data_dir']}/tiger/tiger_2.2_utf8.xml", (0.001, 0.1), character_flag_generators=character_flag_generators, prop_of_tiger_to_use=1.0) # type:ignore

    batch_size = 64

    return (dataset_generator.get_training_dataloader(batch_size=batch_size, shuffle=True),
                    dataset_generator.get_dev_dataloader(batch_size=batch_size, shuffle=False),
                    dataset_generator.get_test_dataloader(batch_size=batch_size, shuffle=False),
                    dataset_generator.character_set,
                    dataset_generator.character_flag_generators,
                    dataset_generator.inverse_word_dict,
                    dataset_generator.inverse_sym_set,
                    dataset_generator.inverse_pos_set,
                    dataset_generator.inverse_morph_dicts
                    )

In [5]:
def dump_required_vars(character_flag_generators, parent_path, round):
    print(f"generating dataset round {round}")
    required_vars = get_required_vars(character_flag_generators)
    pickle.dump(required_vars, open(f"{parent_path}/required_vars_{round}.pkl", "wb"))

In [6]:
pool = multiprocess.Pool(2)

In [7]:
def z(x):
    dump_required_vars(character_flag_generators, "./dataset_rounds", x)


In [8]:
print(pool.map(z, iterations))

generating dataset round 0generating dataset round 5



INFO:model:Parsing dataset from '/home/james/programming/ml/german/german_parser/util/../../data/tiger/tiger_2.2_utf8.xml'...
INFO:model:Parsing dataset from '/home/james/programming/ml/german/german_parser/util/../../data/tiger/tiger_2.2_utf8.xml'...
INFO:model:Parsed 50472 sentences.
INFO:model:Generating trees...
INFO:model:Parsed 50472 sentences.
INFO:model:Generating trees...
INFO:model:48929 (96.94%) trees generated.
INFO:model:Dataset split into 43987 training, 49 dev, and 4893 test trees.
INFO:model:48929 (96.94%) trees generated.
INFO:model:Dataset split into 43987 training, 49 dev, and 4893 test trees.


generating dataset round 6


INFO:model:Parsing dataset from '/home/james/programming/ml/german/german_parser/util/../../data/tiger/tiger_2.2_utf8.xml'...


generating dataset round 1


...

generating dataset round 28


INFO:model:Parsing dataset from '/home/james/programming/ml/german/german_parser/util/../../data/tiger/tiger_2.2_utf8.xml'...
INFO:model:Parsed 50472 sentences.
INFO:model:Generating trees...
INFO:model:48929 (96.94%) trees generated.
INFO:model:Dataset split into 43987 training, 49 dev, and 4893 test trees.


generating dataset round 29


INFO:model:Parsing dataset from '/home/james/programming/ml/german/german_parser/util/../../data/tiger/tiger_2.2_utf8.xml'...
INFO:model:Parsed 50472 sentences.
INFO:model:Generating trees...
INFO:model:48929 (96.94%) trees generated.
INFO:model:Dataset split into 43987 training, 49 dev, and 4893 test trees.


[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [9]:
# (train_dataloader, train_new_words), (dev_dataloader, dev_new_words), (test_dataloader, test_new_words), character_set, character_flag_generators, inverse_word_dict, inverse_sym_dict, inverse_pos_dict, inverse_morph_dicts = pickle.load(open("required_vars.pkl", "rb"))